参考：

1. https://blog.csdn.net/david0611/article/details/81090371
2. https://blog.csdn.net/tommorrow12/article/details/80896331

# Embedding
词嵌入在 pytorch 中非常简单，只需要调用 torch.nn.Embedding(m, n) 就可以了，m 表示单词的总数目，n 表示词嵌入的维度，其实词嵌入就相当于是一个大矩阵，矩阵的每一行表示一个单词。

Pytorch（0.3.1）官网的解释是：一个保存了固定字典和大小的简单查找表。这个模块常用来保存词嵌入和用下标检索它们。模块的输入是一个下标的列表，输出是对应的词嵌入。

torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2, scale_grad_by_freq=False, sparse=False)

个人理解：这是一个矩阵类，里面初始化了一个随机矩阵，矩阵的长是字典的大小，宽是用来表示字典中每个元素的属性向量，向量的维度根据你想要表示的元素的复杂度而定。类实例化之后可以根据字典中元素的下标来查找元素对应的向量。

输入下标0，输出就是embeds矩阵中第0行。

In [3]:
import torch
from torch import nn
from torch.autograd import Variable
# 定义词嵌入
embeds = nn.Embedding(4,5) # 4 个单词，维度 5
# 得到词嵌入矩阵,开始是随机初始化的|
# torch.manual_seed(1)
embeds.weight


Parameter containing:
tensor([[ 1.3689,  0.6877,  0.4720,  0.3814, -0.3354],
        [-0.5924, -0.8084, -0.2716, -0.1118, -0.9694],
        [ 1.0240, -0.6245, -0.3946,  0.7094, -1.0600],
        [ 0.6045, -0.2713,  1.7613, -0.8757, -0.4481]], requires_grad=True)

In [4]:
a = torch.LongTensor([0,3,1])
print(a)
embeds(a)

tensor([0, 3, 1])


tensor([[ 1.3689,  0.6877,  0.4720,  0.3814, -0.3354],
        [ 0.6045, -0.2713,  1.7613, -0.8757, -0.4481],
        [-0.5924, -0.8084, -0.2716, -0.1118, -0.9694]],
       grad_fn=<EmbeddingBackward>)

也可以一次取出2个句子，即读取多个向量。
输入为两个维度(batch的大小，每个batch的单词个数)，输出则在两个维度上加上词向量的大小。

Input: LongTensor (N, W), N = mini-batch, W = number of indices to extract per mini-batch
Output: (N, W, embedding_dim)

In [5]:
embedding = nn.Embedding(10, 3)
# 每批取两组，每组四个单词
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
a = embedding(input) # 输出2*4*3
a[0],a[1]

(tensor([[-0.5175,  0.1208, -0.3148],
         [ 1.0165, -0.2206,  0.4420],
         [-2.0115, -0.8163,  0.5399],
         [-0.7417,  0.3096,  0.2094]], grad_fn=<SelectBackward>),
 tensor([[-2.0115, -0.8163,  0.5399],
         [ 0.3903, -0.0535, -0.1150],
         [ 1.0165, -0.2206,  0.4420],
         [ 1.2313,  0.6669, -0.2497]], grad_fn=<SelectBackward>))

In [6]:
word_to_ix = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5)
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx = Variable(hello_idx)
hello_embed = embeds(hello_idx)
print(hello_embed)

tensor([[ 0.2239, -0.3597, -0.7771, -0.1162,  0.5069]],
       grad_fn=<EmbeddingBackward>)


这就是我们输出的hello这个词的word embedding，下面我们一步一步来解释一下代码

首先我们需要word_to_ix = {'hello': 0, 'world': 1}，每个单词我们需要用一个数字去表示他，这样我们需要hello的时候，就用0来表示它。

接着就是word embedding的定义nn.Embedding(2, 5)，这里的2表示有2个词，5表示5维，其实也就是一个2x5的矩阵，所以如果你有1000个词，每个词希望是100维，你就可以这样建立一个word embedding，nn.Embedding(1000, 100)。如何访问每一个词的词向量是下面两行的代码，注意这里的词向量的建立只是初始的词向量，并没有经过任何修改优化，我们需要建立神经网络通过learning的办法修改word embedding里面的参数使得word embedding每一个词向量能够表示每一个不同的词。

In [7]:
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx = Variable(hello_idx)

接着这两行代码表示得到一个Variable，它的值是hello这个词的index，也就是0。这里要特别注意一下我们需要Variable，因为我们需要访问nn.Embedding里面定义的元素，并且word embeding算是神经网络里面的参数，所以我们需要定义Variable。

hello_embed = embeds(hello_idx)这一行表示得到word embedding里面关于hello这个词的初始词向量，最后我们就可以print出来。

以上我们介绍了word embeding在pytorch里面是如何实现的，下一节我们将介绍如何用word embedding结合n gram模型进行简单的预测。

参考：https://zhuanlan.zhihu.com/p/27119271